In [50]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm

In [47]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 16, 3) 
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(32,64,3)
        self.pool3 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*4*4, 512)
        self.fc2 = nn.Linear(512, 2)
        


    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = F.relu(x)
        x = x.view(-1, 64*4*4)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [48]:
net = Net()
print(net)


import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  
val_size = int(len(X)*VAL_PCT)
print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)
2531


In [30]:
def fwd_pass(X,y,train = False):
    
    if train:
        
        net.zero_grad()
    
    outputs = net(X)
    
    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    
    matches = [torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs,y)]
    
    acc = matches.count(True)/len(matches)
    
    return acc, loss


In [31]:
def test(size = 32):
    
    random_start = np.random.randint(len(test_X)-size)
    X,y = test_X[random_start:random_start+size],test_y[random_start:random_start+size]
    
    with torch.no_grad():
        
        val_acc,val_loss = fwd_pass(X.view(-1,1,50,50),y)
    
    return val_acc,val_loss

In [65]:
def train(batch_size,epochs):
   
    for epoch in range(epochs):
        for i in tqdm(range(0,len(train_X),batch_size)):
            
            X,y = train_X[i:i+batch_size],train_y[i:i+batch_size]
        
            train_acc,train_loss = fwd_pass(X.view(-1,1,50,50),y,train = True)
        
        print("acc,loss: ")
        print(train_acc,train_loss)

In [61]:
train()

  0%|          | 0/228 [00:00<?, ?it/s]

acc,loss: 
0.6363636363636364 tensor(0.2099, grad_fn=<MseLossBackward>)


  0%|          | 1/228 [00:00<00:42,  5.29it/s]

acc,loss: 
0.75 tensor(0.1682, grad_fn=<MseLossBackward>)


100%|██████████| 228/228 [00:42<00:00,  5.37it/s]

acc,loss: 
0.7840909090909091 tensor(0.1473, grad_fn=<MseLossBackward>)
